In [1]:
import pandas as pd
import psycopg2
from pymongo import MongoClient
import re

In [2]:
# Kết nối đến MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['oto_db']
collection = db['oto_collection']

# Truy vấn dữ liệu từ MongoDB
data = list(collection.find())  # Chuyển đổi dữ liệu từ MongoDB sang list of dictionaries
df = pd.DataFrame(data) 
print(df)

                           _id  \
0     67089fd5b7859ab055a9189c   
1     67089fd5b7859ab055a9189d   
2     67089fd5b7859ab055a9189e   
3     67089fd5b7859ab055a9189f   
4     67089fd5b7859ab055a918a0   
...                        ...   
3055  6708cc4a0ff0c9ca3bb3eb2a   
3056  6708cc4a0ff0c9ca3bb3eb2b   
3057  6708cc4a0ff0c9ca3bb3eb2c   
3058  6708cc4a0ff0c9ca3bb3eb2d   
3059  6708cc4a0ff0c9ca3bb3eb2e   

                                         phonenumber  \
0     [ Liên hệ: , ĐT: 0339 048 140 - 0929 488 866 ]   
1                    [ Liên hệ: , ĐT: 0978 290 660 ]   
2                    [ Liên hệ: , ĐT: 0978 290 660 ]   
3                    [ Liên hệ: , ĐT: 0965 551 528 ]   
4                    [ Liên hệ: , ĐT: 0976 226 898 ]   
...                                              ...   
3055  [ Liên hệ: , ĐT: 0938 297 777 - 0916 868 692 ]   
3056                 [ Liên hệ: , ĐT: 0974 677 999 ]   
3057  [ Liên hệ: , ĐT: 0982 020 191 - 0973 533 366 ]   
3058                 [ Liên hệ:

In [22]:
# Hàm xử lý số điện thoại
def xuli_phonenumber(phonenumber):
    if isinstance(phonenumber, list):  # Kiểm tra xem phonenumber có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        phonenumber = ' '.join(phonenumber)
    
    if isinstance(phonenumber, str):  # Nếu là chuỗi, xử lý
        cleaned_phonenumber = re.sub(r'[^0-9-]', '', phonenumber)  # Giữ lại số và dấu '-'
        cleaned_phonenumber = re.sub(r'-+', '-', cleaned_phonenumber)  # Xóa các dấu '-' liên tiếp
        return cleaned_phonenumber.strip('-')  # Xóa '-' đầu và cuối nếu có
    else:
        return None  # Trả về None cho các loại khác
# Áp dụng hàm xử lý cho cột 'phonenumber'
df['phonenumber'] = df['phonenumber'].apply(xuli_phonenumber)
df['phonenumber']


0       0339048140-0929488866
1                  0978290660
2                  0978290660
3                  0965551528
4                  0976226898
                ...          
3055    0938297777-0916868692
3056               0974677999
3057    0982020191-0973533366
3058               0987439968
3059               0797968422
Name: phonenumber, Length: 3060, dtype: object

In [4]:
# Hàm xử lý mã số xe
def xuli_code(code):
    if isinstance(code, list):  # Kiểm tra xem code có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        code = ' '.join(code)
    
    if isinstance(code, str):  # Nếu là chuỗi, xử lý
        cleaned_code = re.sub(r'[^\d]', '', code)  # Giữ lại chỉ các ký tự số
        return cleaned_code
    else:
        return None  # Trả về None cho các loại khác

# Áp dụng hàm xử lý cho cột 'code'
df['code'] = df['code'].apply(xuli_code)
df['code']

0       5857470
1       5780399
2       5743931
3       5869427
4       5837022
         ...   
3055    5784321
3056    5813409
3057    5791797
3058    5854869
3059    5721630
Name: code, Length: 3060, dtype: object

In [5]:
# Hàm xử lý trạng thái xe
def xuli_status(status):
    if isinstance(status, list):  # Kiểm tra xem status có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        status = ' '.join(status)
    
    if isinstance(status, str):  # Nếu là chuỗi, xử lý
        # Giữ lại tất cả các ký tự chữ cái và dấu, bao gồm cả các ký tự đặc biệt trong tiếng Việt
        cleaned_status = re.sub(r'[^0-9a-zA-ZÀ-ỹ\s]', '', status)  # Giữ lại số, chữ (có dấu) và khoảng trắng
        return cleaned_status.strip()  # Xóa khoảng trắng đầu và cuối
    else:
        return None  # Trả về None cho các loại khác

# Áp dụng hàm xử lý cho cột 'status'
df['status'] = df['status'].apply(xuli_status)
df['status'] 

0       Xe cũ
1       Xe cũ
2       Xe cũ
3       Xe cũ
4       Xe cũ
        ...  
3055    Xe cũ
3056    Xe cũ
3057    Xe cũ
3058    Xe cũ
3059    Xe cũ
Name: status, Length: 3060, dtype: object

In [10]:
import re  # Đảm bảo import thư viện re để sử dụng regex

# Hàm xử lý giá
def xuli_price(data):
    if isinstance(data, list):  # Kiểm tra xem data có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        data = ' '.join(data)
        
    if isinstance(data, str):  # Nếu là chuỗi, xử lý
        # Giữ lại tất cả các ký tự chữ cái và số, bao gồm cả các ký tự đặc biệt
        cleaned_data = re.sub(r'[^0-9a-zA-Zà-ỹ\s]', '', data)  # Giữ lại số, chữ (có dấu) và khoảng trắng
        components = cleaned_data.split()  # Tách chuỗi thành các thành phần
        total_value = 0  # Khởi tạo biến tổng giá trị
        
        try:
            # Kiểm tra và cộng giá trị Tỷ
            if 'tỷ' in components:
                ty_index = components.index('tỷ') - 1
                if ty_index >= 0 and components[ty_index].isdigit():  # Kiểm tra có giá trị hợp lệ
                    ty_value = int(components[ty_index]) * 1000
                    total_value += ty_value  # Cộng vào tổng

            # Kiểm tra và cộng giá trị Triệu
            if 'triệu' in components:
                trieu_index = components.index('triệu') - 1
                if trieu_index >= 0 and components[trieu_index].isdigit():  # Kiểm tra có giá trị hợp lệ
                    trieu_value = int(components[trieu_index])
                    total_value += trieu_value  # Cộng vào tổng

            return f"{total_value} triệu" if total_value > 0 else "0 triệu"  # Trả về tổng giá trị
        except (IndexError, ValueError):
            return "0 triệu"  # Giá trị mặc định nếu có lỗi
    else:
        return "0 triệu"  # Nếu data không phải chuỗi, trả về giá trị mặc định

# Áp dụng hàm xử lý cho cột 'price'
df['price'] = df['price'].apply(xuli_price)
df['price']


0       0 triệu
1       0 triệu
2       0 triệu
3       0 triệu
4       0 triệu
         ...   
3055    0 triệu
3056    0 triệu
3057    0 triệu
3058    0 triệu
3059    0 triệu
Name: price, Length: 3060, dtype: object

In [11]:
# Hàm làm sạch địa chỉ
def clean_address(address):
    if isinstance(address, list):  # Kiểm tra xem address có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        address = ' '.join(address)

    if isinstance(address, str):  # Nếu là chuỗi, xử lý
        # Xóa ký tự đặc biệt và dấu ngoặc
        cleaned_address = re.sub(r'[^\w\s]', '', address)  # Xóa dấu ngoặc và ký tự đặc biệt
        cleaned_address = cleaned_address.replace("*", "")  # Xóa dấu *
        cleaned_address = ' '.join(cleaned_address.split())  # Xóa khoảng trắng thừa
        return cleaned_address.strip()  # Xóa khoảng trắng đầu và cuối
    else:
        return None  # Trả về None cho các loại khác

# Áp dụng hàm làm sạch cho cột 'address'
df['address'] = df['address'].astype('str').apply(clean_address)
df['address']

0                    Hạ Yên Quyết Yên Hoà Cầu Giấy Hà Nội
1       393A Nguyễn Xiển Thanh Xuân đối diện trụ cầu V...
2       393A Nguyễn Xiển Thanh Xuân đối diện trụ cầu V...
3                         86 Vũ Đức Thận Long Biên Hà Nội
4                                138 Phạm Văn Đồng Hà Nội
                              ...                        
3055                     Khuất Duy Tiến Thanh Xuân Hà Nội
3056    321 Phạm Văn Đồng Xuân Đỉnh Quận Bắc Từ Liêm H...
3057                   116 Nguyễn Văn Cừ Long Biên Hà Nội
3058                        Quốc lộ 37 Nam Sách Hải Dương
3059         Số 25 đường số 16 Phú Mỹ Hưng Midtown TP HCM
Name: address, Length: 3060, dtype: object

In [12]:
# Hàm làm sạch nội dung
def clean_content(content):
    if isinstance(content, list):  # Kiểm tra xem content có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        content = ' '.join(content)

    if isinstance(content, str):  # Nếu là chuỗi, xử lý
        # Xóa ký tự đặc biệt và dấu ngoặc
        cleaned_content = re.sub(r'[^\w\s]', '', content)  # Xóa dấu ngoặc và ký tự đặc biệt
        cleaned_content = cleaned_content.replace("*", "")  # Xóa dấu *
        cleaned_content = ' '.join(cleaned_content.split())  # Xóa khoảng trắng thừa
        return cleaned_content.strip()  # Xóa khoảng trắng đầu và cuối
    else:
        return None  # Trả về None cho các loại khác

# Áp dụng hàm làm sạch cho cột 'content'
df['content'] = df['content'].astype('str').apply(clean_content)
df['content']

0       Mới về chào bán ngaynNhập Team nguyên xe Mộc H...
1       Sedona GATH 33l sản xuất 2020nXe một chủ duy n...
2       Ford EcoSport Titanium 15 2020 một chủ 3vkm lố...
3       SUZUKI ERTIGA 14 AT 2017n 1 CHỦ TỪ MỚI CHẠY 10...
4       Navara EL 2017 chạy 80000kmnBao khong ngap nuo...
                              ...                        
3055    Xe mới về ạnGLC300 AMG sx 2022 chạy 26000km zi...
3056    Xe đi 6 ngàn km nên như mới ae nhénHuyndai PLI...
3057    HONDA CRV 20AT sx 2015 1 chủ từ đầu biển hà nộ...
3058    Xe đẹp chủ đi rất giữ dìnnChủ là lãnh đạo bí t...
3059    Mercedes_E300AMGn Sản xuất 2022 bản V1n Đăng k...
Name: content, Length: 3060, dtype: object

In [13]:
# Hàm làm sạch thông số
def clean_parameter(parameter):
    if isinstance(parameter, list):  # Kiểm tra xem parameter có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        parameter = ' '.join(parameter)

    if isinstance(parameter, str):  # Nếu là chuỗi, xử lý
        # Xóa ký tự đặc biệt và dấu ngoặc
        cleaned_parameter = re.sub(r'[^\w\s]', '', parameter)  # Xóa dấu ngoặc và ký tự đặc biệt
        cleaned_parameter = cleaned_parameter.replace("*", "")  # Xóa dấu *
        cleaned_parameter = ' '.join(cleaned_parameter.split())  # Xóa khoảng trắng thừa
        return cleaned_parameter.strip()  # Xóa khoảng trắng đầu và cuối
    else:
        return None  # Trả về None cho các loại khác

# Áp dụng hàm làm sạch cho cột 'parameter'
df['parameter'] = df['parameter'].astype('str').apply(clean_parameter)
df['parameter']

0       Xe lắp ráp trong nước màu trắng máy dầu 22 L s...
1       Xe lắp ráp trong nước màu đen máy xăng 33 L số...
2       Xe lắp ráp trong nước màu đỏ máy xăng 15 L số ...
3       Xe nhập khẩu màu xám máy xăng 14 L số tự động ...
4       Xe nhập khẩu màu trắng máy dầu 25 L số tự động...
                              ...                        
3055    Xe lắp ráp trong nước màu đen máy xăng 20 L số...
3056    Xe lắp ráp trong nước màu trắng máy dầu 22 L s...
3057    Xe lắp ráp trong nước màu trắng máy xăng 20 L ...
3058    Xe lắp ráp trong nước màu đen máy xăng 18 L số...
3059    Xe lắp ráp trong nước màu đen máy xăng 20 L số...
Name: parameter, Length: 3060, dtype: object

In [18]:
# Hàm làm sạch cho các cột
def clean_sellername(sellername):
    if isinstance(sellername, list):  # Kiểm tra xem sellername có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        sellername = ' '.join(sellername)

    if isinstance(sellername, str):  # Nếu là chuỗi, xử lý
        cleaned_sellername = re.sub(r'[^\w\s]', '', sellername)  # Xóa dấu ngoặc và ký tự đặc biệt
        cleaned_sellername = cleaned_sellername.replace("*", "")  # Xóa dấu *
        cleaned_sellername = ' '.join(cleaned_sellername.split())  # Xóa khoảng trắng thừa
        return cleaned_sellername.strip()  # Xóa khoảng trắng đầu và cuối
    else:
        return None  # Trả về None cho các loại khác

# Xử lý cho từng cột
df['sellername'] = df['sellername'].astype('str').apply(clean_sellername)
df['sellername']

0                      Mr Phúc
1                   Master Car
2                   Master Car
3                      Xế Auto
4                      Mr Việt
                 ...          
3055                 Hoàng Lân
3056               Độ Vân Auto
3057           Gia Nguyên Auto
3058         Auto Huy Tung 668
3059    Thế Giới Xe Đức CN Sài
Name: sellername, Length: 3060, dtype: object

In [19]:
# Hàm làm sạch cho các cột
def clean_carname(carname):
    if isinstance(carname, list):  # Kiểm tra xem carname có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        carname = ' '.join(carname)

    if isinstance(carname, str):  # Nếu là chuỗi, xử lý
        cleaned_carname = re.sub(r'[^\w\s]', '', carname)  # Xóa dấu ngoặc và ký tự đặc biệt
        cleaned_carname = cleaned_carname.replace("*", "")  # Xóa dấu *
        cleaned_carname = ' '.join(cleaned_carname.split())  # Xóa khoảng trắng thừa
        return cleaned_carname.strip()  # Xóa khoảng trắng đầu và cuối
    else:
        return None  # Trả về None cho các loại khác

# Xử lý cho từng cột
df['carname'] = df['carname'].astype('str').apply(clean_carname)
df['carname']

0           Hyundai SantaFe Cao cấp 22L HTRAC 2022
1                   Kia Sedona 33 GAT Premium 2020
2                Ford EcoSport Titanium 15 AT 2020
3                         Suzuki Ertiga 14 AT 2017
4                  Nissan Navara EL 25 AT 2WD 2017
                           ...                    
3055             Mercedes Benz GLC 300 4Matic 2022
3056    Hyundai Palisade Prestige 22 AT HTRAC 2023
3057                          Honda CRV 20 AT 2015
3058                    Ford Laser GHIA 18 MT 2005
3059           Mercedes Benz E class E300 AMG 2022
Name: carname, Length: 3060, dtype: object

In [20]:
# Hàm làm sạch cho các cột
def clean_city(city):
    if isinstance(city, list):  # Kiểm tra xem city có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        city = ' '.join(city)

    if isinstance(city, str):  # Nếu là chuỗi, xử lý
        cleaned_city = re.sub(r'[^\w\s]', '', city)  # Xóa dấu ngoặc và ký tự đặc biệt
        cleaned_city = cleaned_city.replace("*", "")  # Xóa dấu *
        cleaned_city = ' '.join(cleaned_city.split())  # Xóa khoảng trắng thừa
        return cleaned_city.strip()  # Xóa khoảng trắng đầu và cuối
    else:
        return None  # Trả về None cho các loại khác

# Xử lý cho từng cột
df['city'] = df['city'].astype('str').apply(clean_city)
df['city']

0          Hà Nội
1          Hà Nội
2          Hà Nội
3          Hà Nội
4          Hà Nội
          ...    
3055       Hà Nội
3056       Hà Nội
3057       Hà Nội
3058    Hải Dương
3059       TP HCM
Name: city, Length: 3060, dtype: object

In [21]:
# Hàm làm sạch cho các cột
def clean_id(id):
    if isinstance(id, list):  # Kiểm tra xem id có phải là list không
        # Kết hợp các phần tử thành một chuỗi
        id = ' '.join(id)

    if isinstance(id, str):  # Nếu là chuỗi, xử lý
        cleaned_id = re.sub(r'[^\w\s]', '', id)  # Xóa dấu ngoặc và ký tự đặc biệt
        cleaned_id = cleaned_id.replace("*", "")  # Xóa dấu *
        cleaned_id = ' '.join(cleaned_id.split())  # Xóa khoảng trắng thừa
        return cleaned_id.strip()  # Xóa khoảng trắng đầu và cuối
    else:
        return None  # Trả về None cho các loại khác

# Xử lý cho từng cột
df['_id'] = df['_id'].astype('str').apply(clean_id)
df['_id']

0       67089fd5b7859ab055a9189c
1       67089fd5b7859ab055a9189d
2       67089fd5b7859ab055a9189e
3       67089fd5b7859ab055a9189f
4       67089fd5b7859ab055a918a0
                  ...           
3055    6708cc4a0ff0c9ca3bb3eb2a
3056    6708cc4a0ff0c9ca3bb3eb2b
3057    6708cc4a0ff0c9ca3bb3eb2c
3058    6708cc4a0ff0c9ca3bb3eb2d
3059    6708cc4a0ff0c9ca3bb3eb2e
Name: _id, Length: 3060, dtype: object